# Sentinel-1 and 2 download script
This script allows for acquisition of Sentinel-1 and Sentinel-2 imagery

In [1]:
"""
Script for downloading satellite images from ESA Sentinel Hub server.
Author: Peter Kongstad - Geoscience Master Thesis 2018-2019
E-mail: kongstad@geo.au.dk
"""

'\nScript for downloading satellite images from ESA Sentinel Hub server.\nAuthor: Peter Kongstad - Geoscience Master Thesis 2018-2019\nE-mail: kongstad@geo.au.dk\n'

In [2]:
#%% Imports
import json
import numpy as np
import zipfile
import os
import time
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import datetime
from multiprocessing.pool import ThreadPool
from re import sub

### The API connection is defined below. The geojson file contains the coordinates of a square covering the region of interest. Parameters such as data interval and cloud cover percentage is also defined

In [9]:
#%% API 

# Login information
api = SentinelAPI('kongstad','Delphini1', 'https://scihub.copernicus.eu/dhus')

# Load in the geojson file with the coordinates
region = geojson_to_wkt(read_geojson('geojson/map.ammassalik.geojson'))

# Time span definition
date = ['20160819','20160820']

# Which satellite is it. Outcomment the wrong one. For saving path.
#s=str('S1')
s=str('S2')


# Extraction info
satdata = api.query(region,
                    date=(date[0], date[1]),
                    platformname='Sentinel-2',
                    cloudcoverpercentage=(0,5))

# Converting to a geoJSON
satjson = api.to_geojson(satdata)

# Creating a backup
bu_images_json = satjson

# Listing name of the images being downloaded and file sizes
print("")
print("Executing script for download of Sentinel-2 files")
print("")
print("=================================================")
print("Listing file name and size:")
print("")
total = 0
for n in range(0, np.size(bu_images_json["features"])):
    print("Image name: ",json.dumps(bu_images_json["features"][n]["properties"]["title"]))
    print("File size: ",json.dumps(bu_images_json["features"][n]["properties"]["size"]))
    total += float(sub('[^0-9.]','',json.dumps(bu_images_json["features"][n]["properties"]["size"])))
print("")
print("Found", n+1, "files available for download")
print("Total amount to download: ",round(total, 1), "MB")


Executing script for download of Sentinel-2 files

Listing file name and size:

Image name:  "S2A_MSIL1C_20160819T141012_N0204_R053_T24WVU_20160819T141322"
File size:  "226.19 MB"
Image name:  "S2A_MSIL1C_20160819T141012_N0204_R053_T24WXU_20160819T141322"
File size:  "705.56 MB"

Found 2 files available for download
Total amount to download:  931.8 MB


### Two files are listed, with their respective name and file size. 

In [7]:
#%% Executing this part will initiate download and will take time
print("")    
print("=================================================")
print("Commencing download of", n+1, "files")
print("")
# Setting up download definition
def _download_images(n):
    try:
        print("Downloading file nr.:", n + 1, "of", np.size(bu_images_json["features"]))
        api.download(bu_images_json["features"][n]["properties"]["id"], directory_path='/home/kongstad/Master Thesis/filer/data/satellite_data/'+s+'/packed')
        time.sleep(3) # This is due to avoiding triggerin ESA server overload warning
    except:
        print("Download error detected: ", bu_images_json["features"][n]["properties"]["id"])
        time.sleep(2)
        pass

# Establishing time on download
startTime = datetime.now()

# Counting how many images is scheduled to be downloaded
n = np.size(bu_images_json["features"])

# Initiating the download
multidownload = ThreadPool(processes=2) #Can only download 2 images at once
multidownload.map(_download_images, (filename for filename in range(n)))
multidownload.close()


Commencing download of 2 files



Downloading:   0%|          | 0.00/740M [00:00<?, ?B/s]
Downloading:   2%|▏         | 11.5M/740M [00:00<01:22, 8.88MB/s]
Downloading:   2%|▏         | 13.6M/740M [00:01<01:26, 8.44MB/s]
Downloading:   2%|▏         | 15.7M/740M [00:01<01:16, 9.41MB/s]
Downloading:   1%|▏         | 3.15M/237M [00:01<01:54, 2.04MB/s]
Downloading:   2%|▏         | 4.19M/237M [00:01<01:29, 2.61MB/s]
Downloading:   2%|▏         | 17.8M/740M [00:01<01:24, 8.54MB/s]
Downloading:   3%|▎         | 7.34M/237M [00:01<00:52, 4.35MB/s]
Downloading:   4%|▍         | 9.44M/237M [00:01<00:40, 5.60MB/s]
Downloading:   3%|▎         | 21.0M/740M [00:02<02:04, 5.75MB/s]
Downloading:   3%|▎         | 23.1M/740M [00:02<01:59, 5.99MB/s]
Downloading:   3%|▎         | 24.1M/740M [00:02<01:58, 6.03MB/s]
Downloading:   3%|▎         | 25.2M/740M [00:02<01:52, 6.35MB/s]
Downloading:   4%|▎         | 26.2M/740M [00:03<01:59, 5.97MB/s]
Downloading:   4%|▎         | 27.3M/740M [00:03<02:00, 5.92MB/s]
Downloading:   4%|▍         | 28.3

Downloading:  61%|██████    | 144M/237M [00:24<00:19, 4.84MB/s]
Downloading:  18%|█▊        | 134M/740M [00:25<04:00, 2.52MB/s]
Downloading:  18%|█▊        | 135M/740M [00:25<03:54, 2.58MB/s]
Downloading:  62%|██████▏   | 147M/237M [00:25<00:18, 4.78MB/s]
Downloading:  18%|█▊        | 136M/740M [00:25<03:51, 2.61MB/s]
Downloading:  63%|██████▎   | 149M/237M [00:25<00:18, 4.86MB/s]
Downloading:  19%|█▊        | 137M/740M [00:26<03:44, 2.69MB/s]
Downloading:  64%|██████▎   | 151M/237M [00:26<00:17, 5.00MB/s]
Downloading:  19%|█▊        | 138M/740M [00:26<03:47, 2.65MB/s]
Downloading:  19%|█▉        | 139M/740M [00:27<04:31, 2.21MB/s]
Downloading:  65%|██████▍   | 154M/237M [00:27<00:25, 3.20MB/s]
Downloading:  19%|█▉        | 141M/740M [00:27<04:51, 2.06MB/s]
Downloading:  19%|█▉        | 142M/740M [00:28<05:01, 1.98MB/s]
Downloading:  66%|██████▋   | 157M/237M [00:28<00:26, 3.03MB/s]
Downloading:  19%|█▉        | 143M/740M [00:28<04:58, 2.00MB/s]
Downloading:  19%|█▉        | 144M/740M 

### Files are downloaded and MD5 checksum to confirm file integrity is completed. Next step unzips the files into appropriate folders

In [10]:
#%% Sorting the zipped content into appropriate folders
print("")    
print("=================================================")
print("Unzipping", n+1, "files")
print("")
folders = os.listdir("/home/kongstad/Master Thesis/filer/data/satellite_data/"+s+"/packed")
folders = [f for f in folders if '.zip' in f]
start = time.time()
# Now unwrapping the images and moving them into appropriate folders.
for f in folders:
    try:
        if 'L1C' in f:
            print("Unpacking file:", f)
            with zipfile.ZipFile("/home/kongstad/Master Thesis/filer/data/satellite_data/"+s+"/packed/"+f,"r") as zip_ref:
                zip_ref.extractall("/home/kongstad/Master Thesis/filer/data/satellite_data/"+s+"/unpacked/L1C")
            print("Succesfully unwrapped:",f)
            print("Time elapsed: ",round(time.time()-start,1), "seconds")
            print("")
        elif 'L2A' in f:
            print("Unpacking file:", f)
            with zipfile.ZipFile("/home/kongstad/Master Thesis/filer/data/satellite_data/"+s+"/packed/"+f,"r") as zip_ref:
                zip_ref.extractall("/home/kongstad/Master Thesis/filer/data/satellite_data/"+s+"/L2C")
            print("Succesfully unwrapped:",f)
            print("Time elapsed: ",round(time.time()-start,1), "seconds")
            print("")
    except:
        print("Unwrap error with file:", f)
        pass

print("=========================================")
print("Download and extraction of files complete")
print("=========================================")
print("")


Unzipping 2 files

Unpacking file: S2A_MSIL1C_20180720T141011_N0206_R053_T24WVT_20180720T160525.zip
Succesfully unwrapped: S2A_MSIL1C_20180720T141011_N0206_R053_T24WVT_20180720T160525.zip
Time elapsed:  1.3 seconds

Unpacking file: S2A_MSIL1C_20160720T141012_N0204_R053_T24WXT_20160720T141008.zip
Succesfully unwrapped: S2A_MSIL1C_20160720T141012_N0204_R053_T24WXT_20160720T141008.zip
Time elapsed:  2.5 seconds

Unpacking file: S2A_MSIL1C_20160720T141012_N0204_R053_T24WWU_20160720T141008.zip
Succesfully unwrapped: S2A_MSIL1C_20160720T141012_N0204_R053_T24WWU_20160720T141008.zip
Time elapsed:  4.6 seconds

Unpacking file: S2A_MSIL1C_20160720T141012_N0204_R053_T24WWT_20160720T141008.zip
Succesfully unwrapped: S2A_MSIL1C_20160720T141012_N0204_R053_T24WWT_20160720T141008.zip
Time elapsed:  6.1 seconds

Unpacking file: S2B_MSIL1C_20180725T140959_N0206_R053_T24WVT_20180725T180250.zip
Succesfully unwrapped: S2B_MSIL1C_20180725T140959_N0206_R053_T24WVT_20180725T180250.zip
Time elapsed:  7.4 seco

### All files are downloaded and ready to use